In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [11]:
# Ruta de la carpeta logs, que está un nivel arriba del notebook
logs_dir = os.path.join(os.getcwd(), '..', 'logs')   # Sube un nivel y entra a logs
logs_dir = os.path.abspath(logs_dir)                 # Convierte a ruta absoluta

In [ ]:
print("Buscando en:", logs_dir)
print("¿Existe 'logs'? ->", os.path.isdir(logs_dir))

Buscando en: c:\Users\jdvil\Documents\Python\Practicas\Juego_Vida\logs
¿Existe 'logs'? -> True


In [ ]:
if os.path.isdir(logs_dir):
    csv_files = [f for f in os.listdir(logs_dir) if f.endswith('.csv')]
    print("Archivos CSV encontrados:", csv_files)
    if csv_files:
        csv_files.sort(key=lambda f: os.path.getmtime(os.path.join(logs_dir, f)), reverse=True)
        csv_path = os.path.join(logs_dir, csv_files[0])
        print("Cargando archivo:", csv_files[0])
        df = pd.read_csv(csv_path)
        print(df.head())
    else:
        print("No hay archivos CSV en la carpeta logs.")
else:
    print("La carpeta 'logs' no existe en la ubicación esperada.")

Archivos CSV encontrados: ['civilizaciones_log_2025-06-23_175415.csv']
Cargando archivo: civilizaciones_log_2025-06-23_175415.csv
   ciclo civilizacion personalidad estrategia  fuerza  tecnologia  religion  \
0      1         Civ1  militarista       hawk      45         0.0       0.0   
1      1         Civ2       halcon       dove      45         0.0       0.0   
2      1         Civ3  equilibrado       hawk      45         0.0       0.0   
3      2         Civ1  militarista       hawk      70         1.2       0.0   
4      2         Civ2       halcon       dove      77         0.0       0.0   

   militar  complejidad  celdas  plantasTec  iglesias  cuarteles  
0      0.0          9.8      30           0         0          0  
1      0.0         10.2      30           0         0          0  
2      0.0          5.8      30           0         0          0  
3      0.0         11.6      45           1         0          0  
4      0.0          7.4      51           0         0       

In [14]:
df.head()

,ciclo,civilizacion,personalidad,estrategia,fuerza,tecnologia,religion,militar,complejidad,celdas,plantasTec,iglesias,cuarteles
0,1,Civ1,militarista,hawk,45,0.0,0.0,0.0,9.8,30,0,0,0
1,1,Civ2,halcon,dove,45,0.0,0.0,0.0,10.2,30,0,0,0
2,1,Civ3,equilibrado,hawk,45,0.0,0.0,0.0,5.8,30,0,0,0
3,2,Civ1,militarista,hawk,70,1.2,0.0,0.0,11.6,45,1,0,0
4,2,Civ2,halcon,dove,77,0.0,0.0,0.0,7.4,51,0,0,0


In [15]:
# --- Crear carpeta para guardar imágenes ---
os.makedirs('kpi_imgs', exist_ok=True)
img_paths = []

In [ ]:
# 1. Crecimiento territorial (KPI 1)
plt.figure()
df_pivot = df.pivot(index='ciclo', columns='civilizacion', values='celdas')
df_pivot.plot(figsize=(8,5), title="Crecimiento territorial (celdas ocupadas)", ylabel="Celdas")
plt.xlabel("Ciclo")
plt.tight_layout()
img1 = 'kpi_imgs/kpi1_crecimiento_celdas.png'
plt.savefig(img1)
img_paths.append(img1)
plt.close()

C:\Users\jdvil\AppData\Local\Temp\ipykernel_9348\3849198276.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['fuerza_x_celda'] = final['fuerza'] / final['celdas']


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
# 2. Especialización en edificios (KPI 2)
plt.figure()
final = df[df['ciclo'] == df['ciclo'].max()]
final.set_index('civilizacion')[['plantasTec','iglesias','cuarteles']].plot(kind='bar', stacked=True, figsize=(8,5))
plt.title("Edificios construidos por civilización (final)")
plt.ylabel("Cantidad")
plt.tight_layout()
img2 = 'kpi_imgs/kpi2_edificios.png'
plt.savefig(img2)
img_paths.append(img2)
plt.close()


In [ ]:

# 3. Eficiencia: Fuerza por celda (KPI 3)
plt.figure()
final['fuerza_x_celda'] = final['fuerza'] / final['celdas']
final.set_index('civilizacion')['fuerza_x_celda'].plot(kind='bar', color='teal')
plt.title("Eficiencia: Fuerza por celda (final)")
plt.ylabel("Fuerza/celda")
plt.tight_layout()
img3 = 'kpi_imgs/kpi3_fuerza_x_celda.png'
plt.savefig(img3)
img_paths.append(img3)
plt.close()


In [ ]:

# 4. Estrategia dominante por personalidad (KPI 4)
plt.figure()
pers = df.groupby('personalidad')['fuerza'].max().sort_values()
pers.plot(kind='barh', color='purple')
plt.title("Fuerza máxima por personalidad")
plt.xlabel("Fuerza máxima")
plt.tight_layout()
img4 = 'kpi_imgs/kpi4_fuerza_personalidad.png'
img_paths.append(img4)
plt.savefig(img4)
plt.close()


In [ ]:

# 5. Eventos clave: saltos grandes de fuerza (KPI 5)
plt.figure()
for civ in df['civilizacion'].unique():
    fuerza = df[df['civilizacion']==civ][['ciclo','fuerza']]
    fuerza['delta'] = fuerza['fuerza'].diff().fillna(0)
    plt.plot(fuerza['ciclo'], fuerza['delta'], label=civ)
plt.title("Saltos de fuerza por ciclo (eventos clave)")
plt.xlabel("Ciclo")
plt.ylabel("Cambio en fuerza respecto al ciclo anterior")
plt.legend()
plt.tight_layout()
img5 = 'kpi_imgs/kpi5_eventos_clave.png'
plt.savefig(img5)
img_paths.append(img5)
plt.close()


In [ ]:

# 6. Comparación de personalidades (KPI 6)
plt.figure()
pers_prom = df.groupby('personalidad').agg({
    'fuerza':'mean', 'celdas':'mean', 'complejidad':'mean'
}).sort_values('fuerza', ascending=False)
pers_prom[['fuerza','celdas','complejidad']].plot(kind='bar', figsize=(9,5))
plt.title("Promedio de fuerza, celdas y complejidad por personalidad")
plt.tight_layout()
img6 = 'kpi_imgs/kpi6_promedio_personalidad.png'
plt.savefig(img6)
img_paths.append(img6)
plt.close()


In [ ]:

# 7. Análisis temporal: evolución de fuerza (KPI 7)
plt.figure()
for civ in df['civilizacion'].unique():
    plt.plot(df[df['civilizacion']==civ]['ciclo'], df[df['civilizacion']==civ]['fuerza'], label=civ)
plt.title("Evolución de fuerza por ciclo")
plt.xlabel("Ciclo")
plt.ylabel("Fuerza")
plt.legend()
plt.tight_layout()
img7 = 'kpi_imgs/kpi7_evolucion_fuerza.png'
plt.savefig(img7)
img_paths.append(img7)
plt.close()